In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
import random
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.utils import to_categorical

In [2]:
vocab_sizeee=1000
batch_size=32
max_len=15

In [3]:
data=pd.read_csv("F:/open_zip/flickr1k/captions.csv")

In [4]:
data.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [5]:
data["caption"]=data["caption"].apply(lambda line:word_tokenize(line.lower()))

In [6]:
data.head()

,image,caption
0,1000268201_693b08cb0e.jpg,"[a, child, in, a, pink, dress, is, climbing, u..."
1,1000268201_693b08cb0e.jpg,"[a, girl, going, into, a, wooden, building, .]"
2,1000268201_693b08cb0e.jpg,"[a, little, girl, climbing, into, a, wooden, p..."
3,1000268201_693b08cb0e.jpg,"[a, little, girl, climbing, the, stairs, to, h..."
4,1000268201_693b08cb0e.jpg,"[a, little, girl, in, a, pink, dress, going, i..."


In [7]:
words=[]

for line in data["caption"]:
    words.extend(line)

In [8]:
len(words)

59960

In [9]:
wordss,count=np.unique(words,return_counts=True)

In [10]:
len(wordss)

3290

In [11]:
vocab=wordss[np.argsort(count)[-vocab_sizeee:]]

In [12]:
len(vocab)

1000

In [13]:
bad=stopwords.words("english") + list(string.punctuation)

In [14]:
len(bad)

211

In [15]:
vocab_si=[word for word in vocab if word not in bad]

In [16]:
vocab_size=len(vocab_si)

In [17]:
vocab_size

927

In [45]:
r_vocab=dict(zip(vocab_si,np.arange(1,vocab_size+1)))

In [46]:
rev_vocab=dict(zip(np.arange(1,vocab_size+1),vocab_si))

In [47]:
sent=data.caption[2000]

In [48]:
" ".join(sent[1:-1])

'climber in an orange helmet is ascending attached to a rope whilst climbing a rock face'

In [50]:
" ".join(word for word in sent[1:-1] if word in r_vocab)

'climber orange helmet attached rope whilst climbing rock face'

In [51]:
#line = random.choice(data.caption)
#line = [vocab[word] for word in line if word in vocab]

#line

In [55]:
def data_gen(lines, batch_size=32):
    
    n = 0
    
    X, Y = [], []
    
    while True:
        
        line = random.choice(lines)
        line = [r_vocab[word] for word in line if word in r_vocab]
        if len(line) > 4:
            index = random.randint(0, len(line)-1)
            X.append(line[:index])
            Y.append(to_categorical(line[index], num_classes=vocab_size+1))
            n += 1

            if n % batch_size == 0:
                X_mod = pad_sequences(X, maxlen=max_len)
                y_mod = np.array(Y)
                X, Y = [], []
                yield [X_mod, y_mod]


In [56]:
dat=data_gen(data["caption"])

In [57]:
datas=next(dat)

In [58]:
a=datas[0][0]
b=datas[1][0]

In [59]:
[rev_vocab[word] for word in a if word in rev_vocab]

['little', 'boy']

In [60]:
[rev_vocab[word] for word in b if word in rev_vocab]

['gas']

In [61]:
datas[0].shape

(32, 15)

In [62]:
datas[1].shape

(32, 928)

In [63]:
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model

In [64]:
in_layer=Input(shape=(max_len,))
embad=Embedding(input_dim=vocab_size+1 , output_dim=10)(in_layer)
rnn=LSTM(units=500)(embad)
d1=Dense(units=100,activation="tanh")(rnn)
d2=Dense(units=10,activation="tanh")(d1)
out=Dense(units=vocab_size+1,activation="softmax")(d2)

In [65]:
model=Model(inputs=in_layer,outputs=out)

In [66]:
model.compile(    optimizer='rmsprop',
    loss="categorical_crossentropy")

In [67]:
gen=data_gen(data.caption,batch_size=2000)

for i in range(5):
    dat=next(gen)
    model.fit(dat[0],dat[1])

Train on 2000 samples
2000/2000 [==============================] - 14s 7ms/sample - loss: 6.6269
Train on 2000 samples
2000/2000 [==============================] - 11s 5ms/sample - loss: 6.2711
Train on 2000 samples
2000/2000 [==============================] - 11s 5ms/sample - loss: 6.0932
Train on 2000 samples
2000/2000 [==============================] - 11s 5ms/sample - loss: 5.8935
Train on 2000 samples
2000/2000 [==============================] - 10s 5ms/sample - loss: 5.8389


In [78]:
sele=data["caption"][150][:8]

In [79]:
sele

['a', 'boy', 'bites', 'hard', 'into', 'a', 'treat', 'while']

In [80]:
final=[r_vocab[w] for w in sele if w in r_vocab]

In [81]:
final

[924]

In [82]:
X=pad_sequences([final],maxlen=max_len)

In [83]:
X

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0, 924]])

In [84]:
model.predict(X).argmax(axis=1)

array([927], dtype=int64)

In [85]:
rev_vocab[927]

'dog'